### Dense Networks Experiments

This first notebook is to explore the effectiveness of a fully connected neural network to classify tags. The testing will be as follows:

1. Test data which has been subsampled 


In [1]:
import tensorflow as tf
from pydst.rate_limiters import RateLimited 
from pydst.dataproviders import DataProvider
from IPython.display import display, HTML
import datetime
import os
import numpy as np

In [2]:
def show_graph(graph_def, frame_size=(900, 600)):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:{height}px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(height=frame_size[1], data=repr(str(graph_def)), id='graph'+timestamp)
    iframe = """
        <iframe seamless style="width:{width}px;height:{height}px;border:0" srcdoc="{src}"></iframe>
    """.format(width=frame_size[0], height=frame_size[1] + 20, src=code.replace('"', '&quot;'))
    display(HTML(iframe))

In [3]:
def dense(name, inputs, units, fn):
    with tf.variable_scope(name):
        print(inputs.shape[1])
        output = tf.layers.dense(inputs=inputs, units=units, name=name, activation=fn, 
                                 kernel_initializer=None) #tf.truncated_normal_initializer())
        return output
#(stddev=(2./(inputs.shape[1]+units))**0.5)

def mlp_graph(which_set, rng=None):
    
    graph = tf.Graph()
    outputs = {}
    errors = {}
    accs = {}
    
    with graph.as_default():

        with tf.device('/cpu:0'):
            dataprovider = DataProvider(graph=graph, which_set=which_set, batch_size=10, 
                                        down_sample=20, target_size=4,rng=rng)
                                 
            inputs, targets = dataprovider.get_data()
            
        with tf.device('/cpu:0'):
            name = 'layer-1'
            outputs[name] = dense(name, inputs, 10000, tf.nn.relu)                                            

            name = 'layer-2'
            outputs[name] = dense(name, outputs['layer-1'], 6000, tf.nn.relu)

            name = 'layer-3'
            outputs[name] = dense(name, outputs['layer-2'], 3000, tf.nn.relu)

            name = 'layer-4'
            outputs[name] = dense(name, outputs['layer-3'], 2000, tf.nn.relu)

            name = 'layer-5'
            outputs[name] = dense(name, outputs['layer-4'], 500, tf.nn.relu)

            name = 'layer-6'
            outputs[name] = dense(name, outputs['layer-5'], 100, tf.nn.relu)

            name = 'layer-7'
            outputs[name] = dense(name, outputs['layer-6'], 50, tf.nn.relu)

            name = 'layer-8'
            outputs[name] = dense(name, outputs['layer-7'], 10, tf.nn.relu)

            name = 'layer-9'
            outputs[name] = dense(name, outputs['layer-8'], 4, tf.nn.relu)

        last_output = outputs['layer-9']
        name = 'error-l9'
        with tf.variable_scope(name):
            errors[name] = tf.losses.sigmoid_cross_entropy(multi_class_labels=targets, logits=last_output)
        tf.summary.scalar(name, errors[name])
        train_step = tf.train.AdamOptimizer().minimize(errors[name])
        
        name = 'accuracy-l9'
        with tf.variable_scope(name):
            accs[name] = tf.reduce_mean(tf.cast(
                tf.equal(tf.argmax(last_output, 1), tf.argmax(targets, 1)), tf.float32))
        tf.summary.scalar(name, accs[name])


        summary_op = tf.summary.merge_all()
        saver = tf.train.Saver()
        
        sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=8))
        
    return sess, graph, summary_op, train_step, dataprovider, saver, inputs, targets

In [ ]:
with tf.variable_scope("model_t1") as scope:
    [train_sess, train_graph, train_summary, train_step, trainData, trainSaver, 
                                                                tinputs, ttargets] = mlp_graph('train')
    
#show_graph(train_graph)

timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
writer = tf.summary.FileWriter(os.path.join('logs', 'dense_exp', timestamp, 'train'), graph=train_graph)

with train_graph.as_default():

    train_sess.run(tf.global_variables_initializer())
    trainData.enable(train_sess)

    max_iters = trainData.max_num_batches()

    print('Starting iterations')
    for epoch in range(30):

        run_error = 0.
        run_acc = 0.
        print('Epoch: {}'.format(epoch))
        for idx in range(max_iters):

            [_, summary_results] = train_sess.run([train_step, train_summary])          
            writer.add_summary(summary_results, epoch * max_iters + idx)

    trainData.disable(train_sess)

23300
10000
6000
3000
2000
500
100
50
10
Starting iterations
Epoch: 0
Epoch: 1
